In [1]:
from pyiem.network import Table as NetworkTable
import pandas as pd
from pandas.io.sql import read_sql
from pyiem.plot import MapPlot
import numpy as np
from tqdm import tqdm
from pyiem.meteorology import windchill
from pyiem.datatypes import temperature, speed
from pyiem.util import get_dbconn

Populating the interactive namespace from numpy and matplotlib


In [2]:
ASOS = get_dbconn('asos')

MESOSITE = get_dbconn('mesosite')
mcursor = MESOSITE.cursor()

mcursor.execute("""
    SELECT id, network, st_x(geom) as lon, st_y(geom) as lat from stations where country = 'US' and 
    network ~* 'ASOS' and state not in ('AK', 'HI') and
    archive_begin < '1973-01-01' and archive_end is null and length(id) = 3""")

rows = []
progress = tqdm(mcursor, total=mcursor.rowcount)
for row in progress:
    progress.set_description(row[0])
    # Figure out what our archive coverage is, 41 years would be perfect
    # https://leafo.net/guides/postgresql-calculating-percentile.html#calculating-all-percentiles
    df = read_sql("""
        with data as (
        SELECT date_trunc('hour', valid) as hr, max(feel)::int as val from alldata
        where station = %s and valid > '1973-01-01' and feel is not null and feel < 200
        and extract(month from valid) in (4, 5, 6, 7, 8, 9)
        and report_type = 2 GROUP by hr)
        select percentile_disc(0.990) within group (ORDER by val) from data
    """, ASOS, params=(row[0],) )

    # 12 coldest hours over 6 winter months is 99.8th percentile
    # val = np.percentile(df['val'].values, np.arange(98.5, 100.1, 0.1))
    rows.append(dict(sid=row[0], lat=row[3], lon=row[2],
                     val=df.iat[0, 0], network=row[1]))

df = pd.DataFrame(rows)
df.set_index('sid', inplace=True)
df['useme'] = True

CRP: 100%|██████████| 690/690 [3:00:49<00:00, 15.72s/it]  


In [3]:
df = pd.read_csv('heat_percentiles.csv')
#df.at['3A1', 'useme'] = False
#df.at['87Q', 'useme'] = False
print(df)

     sid        lat        lon  val  network  useme
0    MCN  32.692850 -83.649210  103  GA_ASOS   True
1    MCO  28.429390 -81.308990  102  FL_ASOS   True
2    BAK  39.261920 -85.896330   96  IN_ASOS   True
3    BTV  44.470000 -73.150000   90  VT_ASOS   True
4    CVG  39.043060 -84.671670   97  KY_ASOS   True
..   ...        ...        ...  ...      ...    ...
685  EYW  24.556111 -81.759556  103  FL_ASOS   True
686  PIT  40.491470 -80.232860   91  PA_ASOS   True
687  SHV  32.447200 -93.824400  105  LA_ASOS   True
688  JAN  32.320000 -90.080000  104  MS_ASOS   True
689  CRP  27.773060 -97.512780  107  TX_ASOS   True

[690 rows x 6 columns]


In [4]:
df2 = df[df['useme'] == True]
for percentile in [99.0, ]:
    # percentile = 100. - (p * 0.1)
    values = df2['val']
    hours = 182. * 24. * ((100. - percentile) / 100.)
    m = MapPlot(sector='conus',
                title=("%.1fth Percentile Hottest Heat Index Temperature (April thru September)"
                       ) % (percentile,),
                subtitle=("based on longterm automated stations hourly METARs, "
                          "%.1f is ~%.0f hours out of six months"
                         ) % (percentile, hours))
    m.contourf(np.array(df2.lon), np.array(df2.lat), np.array(values), np.arange(80, 111, 5), units='F',
              cmap=plt.get_cmap('jet'))
    m.plot_values(df2.lon, df2.lat, values, fmt='%.0f', labelbuffer=15)
    m.postprocess(filename='hdx_%.0f.png' % (percentile * 10,))
    m.close()